In [ ]:
import psutil
import pandas as pd
import re
import requests
import json
import gmaps
import gmaps.datasets

In [ ]:
googleAPIKey = '' #add google api key here
token_str = '' #replace xxxxx with ipInfo api token

In [ ]:
privIP = re.compile("^(?:10|127|172\.(?:1[6-9]|2[0-9]|3[01])|192\.168)\..*")
priv6 = re.compile("::1")
url_str = 'https://ipinfo.io/'
netstat = pd.DataFrame(columns=['status', 'pid', 'local_ip', 'local_port', 'remote_ip', 'remote_port'])
con = psutil.net_connections()
s_list = []
for item in con:
    try:
        if not re.search(privIP, item.raddr.ip) and not re.search(priv6, item.raddr.ip):
            data = {'status':item.status, 'pid':item.pid, 'local_ip':item.laddr.ip, 'local_port': item.laddr.port, 'remote_ip':item.raddr.ip, 'remote_port':item.raddr.port}
            s = pd.Series(data)
            s_list.append(s)
    except:
        pass
netstat = pd.DataFrame(s_list)
data = []
for ip in netstat.remote_ip:
    res = (json.loads(requests.get(url_str + ip + '?token=' + token_str).text))
    data.append(res)
ipInfo = pd.json_normalize(data)
# Iterate over all running process
data = []
for proc in psutil.process_iter():
    try:
        res = {'process_name':proc.name(), 'pid':proc.pid}
        data.append(res)
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass
processes = pd.json_normalize(data)
mergedf = pd.merge(netstat, ipInfo.rename(columns={'ip':'remote_ip'}), how='outer')
mergedf = pd.merge(mergedf, processes, how='outer')
processConnections = mergedf[mergedf.remote_ip.notnull()]
gmaps.configure(api_key=googleAPIKey)
locList = list(mergedf['loc'].dropna())

locations = []
for i, loc in enumerate(locList):
    listloc = loc.split(',')
    lat = float(listloc[0])
    lon = float(listloc[1])
    locations.append(tuple([lat, lon]))

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
fig

In [ ]:
pd.set_option('display.max_rows', None)
processConnections